In [35]:
from src.data_structures import Instance
from src.data_structures.features import *
from src.solvers.collection import SolverCollection
import torch
from torch import nn
import os
from pathlib import Path
import sys
torch.set_default_tensor_type(torch.DoubleTensor)


In [36]:
#Training data (Instancias ya solucionadas)
def encode_bool_vec(a):
    indices = a.nonzero()
    a-= 1
    indices.T[0]
    a[indices] = 1
    return a

def decode_bool_vec(a):
    a = a +  1
    indices = a.nonzero()
    a[indices] = 1
    return a//2



max_size = 1500
training_data_folder = Path("training_data")
expand_vector = lambda x,instance: torch.cat([x,torch.zeros(max_size-instance.n_items)])
get_y = lambda instance,sol: expand_vector(torch.tensor(sol),instance)

In [37]:
#Features
#Debo generar un vector x por cada feature, y cada vector x debe expandirse,
#y luego concatenar todos los vectores x expandidos por lo que la dimension de entrada
#sera #features*n_items.

#Esto se hace por cada instancia

features: list[ItemBatchFeature] = [
    Budget,
    ProfitOverBudget,
    LowerCostOverBudget,
    UpperCostOverBudget,
    #IsInContSol,
    ]

def gen_x(features: list[ItemBatchFeature],instance: Instance):
    evaluated_features = []
    for feature in features:
        x_feature = torch.tensor(feature.batch_evaluate(instance))
        x_feature = expand_vector(x_feature,instance)
        evaluated_features.append(x_feature)
    return torch.cat(evaluated_features)

In [65]:
class Net(nn.Module):
    def __init__(self, entrada,salida):
        super(Net, self).__init__()

        hidden_size = 200

        self.many = nn.Sequential(
            nn.Linear(entrada, hidden_size),
            nn.Linear(hidden_size,hidden_size),
            nn.Linear(hidden_size,salida),
            nn.Tanh()
        )

    def forward(self, x):   
        x = self.many(x)
        return x

In [68]:
instances = map(Instance.from_file,training_data_folder.iterdir()).__iter__()
net = Net(len(features)*max_size,max_size)
criterion = nn.L1Loss()
learning_rate = 0.01
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)

In [73]:
#Training
for i,instance in enumerate(instances):
    optimizer.zero_grad()
    total_loss = 0
    x = gen_x(features,instance)
    y_pred = net(x)
    batch_loss = criterion(y_pred,encode_bool_vec(get_y(instance,instance.optimal_solution)) )
    batch_loss.backward()
    optimizer.step()
    total_loss += batch_loss.item()
    sys.stdout.write(f'\rEpoch {instance}, CorrectedLoss {total_loss}')
    sys.stdout.flush()


Epoch Instance(693,93,#-5976011006941083271), CorrectedLoss 0.5460799433867307

Epoch Instance(989,42,#8026829103407780265), CorrectedLoss 0.67794576692872343

In [74]:
test_instance = Instance.generate(1500,50)
solution = SolverCollection.gurobi_optimal(test_instance)

In [83]:
with torch.no_grad():
    test_x = gen_x(features,test_instance)
    test_y = net(test_x)

In [84]:
with torch.no_grad():
    loss = criterion(test_y,expand_vector(torch.tensor(solution.sol),test_instance))
    print(loss)

tensor(1.4542)
